In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import matplotlib

from finrl import config
from finrl import config_tickers
from finrl.meta.preprocessor.yahoodownloader import YahooDownloader
from finrl.meta.preprocessor.preprocessors import FeatureEngineer, data_split
from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from finrl.plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from finrl.main import check_and_make_directories
from pprint import pprint
from stable_baselines3.common.logger import configure

from finrl.config import (
    DATA_SAVE_DIR,
    TRAINED_MODEL_DIR,
    TENSORBOARD_LOG_DIR,
    RESULTS_DIR,
    INDICATORS,
    TRAIN_START_DATE,
    TRAIN_END_DATE,
    TEST_START_DATE,
    TEST_END_DATE,
    TRADE_START_DATE,
    TRADE_END_DATE,
)

from finrl.config_tickers import DOW_30_TICKER

import talib

from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier 

from sklearn.metrics import roc_auc_score, f1_score

%matplotlib inline

/Users/stureiko/miniforge3/envs/finrl/lib/python3.10/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


Создаем служебные каталоги

In [2]:
check_and_make_directories([DATA_SAVE_DIR, TRAINED_MODEL_DIR, TENSORBOARD_LOG_DIR, RESULTS_DIR])

Загружаем исходные данные

In [3]:
DOW_30_TICKER

['AXP',
 'AMGN',
 'AAPL',
 'BA',
 'CAT',
 'CSCO',
 'CVX',
 'GS',
 'HD',
 'HON',
 'IBM',
 'INTC',
 'JNJ',
 'KO',
 'JPM',
 'MCD',
 'MMM',
 'MRK',
 'MSFT',
 'NKE',
 'PG',
 'TRV',
 'UNH',
 'CRM',
 'VZ',
 'V',
 'WBA',
 'WMT',
 'DIS',
 'DOW']

In [4]:
TRAIN_START_DATE = '2009-01-01'
TRAIN_END_DATE = '2019-01-01'
TEST_START_DATE = '2019-01-01'
TEST_END_DATE = '2021-01-01'

# загружаем данные
df = YahooDownloader(start_date = TRAIN_START_DATE,
                     end_date = TEST_END_DATE,
                     ticker_list = DOW_30_TICKER).fetch_data()

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Shape of DataFrame:  (88061, 8)


Преобразуем дату и сортируем датасет по дате и бумагам

In [5]:
# преобразуем дату
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d')

# Отсортируем данные по дате и коду акции
df.sort_values(['date','tic'],ignore_index=True).head()

,date,open,high,low,close,volume,tic,day
0,2009-01-02,3.067143,3.251429,3.041429,2.740172,746015200,AAPL,4
1,2009-01-02,58.590000,59.080002,57.750000,42.107327,6547900,AMGN,4
2,2009-01-02,18.570000,19.520000,18.400000,15.098145,10955700,AXP,4
3,2009-01-02,42.799999,45.560001,42.779999,33.941093,7010200,BA,4
4,2009-01-02,44.910000,46.980000,44.709999,30.837601,7117200,CAT,4


Выберем только бумаги Apple

In [6]:
data = df.loc[df['tic'] == 'AAPL']
data.drop(['tic', 'day'], axis=1, inplace=True)
data.head()

,date,open,high,low,close,volume
0,2009-01-02,3.067143,3.251429,3.041429,2.740172,746015200
29,2009-01-05,3.327500,3.435000,3.311071,2.855818,1181608400
58,2009-01-06,3.426786,3.470357,3.299643,2.808715,1289310400
87,2009-01-07,3.278929,3.303571,3.223571,2.748023,753048800
116,2009-01-08,3.229643,3.326786,3.215714,2.799052,673500800


## Технический анализ

Будем формировать показатели с использованием библиотеки TA-Lib (https://ta-lib.org)

### Простое скользящее среднее (Simple Moving Averages - SMA)

Скользящие средние - один из самых популярных технических индикаторов, используемых для сглаживания ценовых колебаний и выявления трендов. Они могут рассчитываться с использованием различных временных периодов, таких как краткосрочный (10 дней), среднесрочный (50 дней) и долгосрочный (200 дней).

In [7]:
data["sma_5"] = talib.SMA(data["close"], timeperiod=5)
data["sma_10"] = talib.SMA(data["close"], timeperiod=10)
data["sma_15"] = talib.SMA(data["close"], timeperiod=15)
data["sma_50"] = talib.SMA(data["close"], timeperiod=50)
data["sma_200"] = talib.SMA(data["close"], timeperiod=200)
data["sma_ratio_5_15"] = data['sma_15'] / data['sma_5']
data["sma_ratio_10_50"] = data['sma_50'] / data['sma_15']
data['sma_5_Volume'] = talib.SMA(data["volume"], timeperiod=5)
data['sma_10_Volume'] = talib.SMA(data["volume"], timeperiod=10)
data['sma_15_Volume'] = talib.SMA(data["volume"], timeperiod=15)
data['sma_50_Volume'] = talib.SMA(data["volume"], timeperiod=50)
data["sma_Volume_ratio_5_15"] = data['sma_5_Volume'] / data['sma_15_Volume']
data["sma_Volume_ratio_10_50"] = data['sma_10_Volume'] / data['sma_50_Volume']

### Полосы Боллинджера (Bollinger Bands)

Полосы Боллинджера используются для измерения волатильности актива и выявления условий перекупленности и перепроданности. Полосы состоят из скользящей средней (обычно 20-дневной скользящей средней) и двух линий стандартного отклонения выше и ниже скользящей средней.

In [8]:
# Calculate the Bollinger Bands
data["upper_band"], data["middle_band"], data["lower_band"] = talib.BBANDS(data["close"], timeperiod=20)

### Индекс относительной силы (Relative Strength Index - RSI)

RSI - это индикатор импульса, который сравнивает величину недавних достижений с недавними потерями, чтобы определить условия перекупленности и перепроданности. Обычно он используется с 14-дневным периодом.

In [9]:
# Calculate the relative strength index (RSI)
data["RSI"] = talib.RSI(data["close"], timeperiod=14)

### Дивергенция сходимости скользящих средних (Moving Average Convergence Divergence - MACD)

MACD - это индикатор следования за трендом, который измеряет разницу между краткосрочной скользящей средней и долгосрочной скользящей средней. Обычно он используется с 12-дневной и 26-дневной скользящей средней, а также с 9-дневной сигнальной линией.

In [10]:
# Calculate the MACD
data["macd"], data["macd_signal"], data["macd_hist"] = talib.MACD(data["close"], fastperiod=12, slowperiod=26, signalperiod=9)

### Стохастический осциллятор (Stochastic Oscillator)

Стохастический осциллятор - это индикатор импульса, который сравнивает цену закрытия актива с его ценовым диапазоном за определенный период. Обычно он используется с 5-и и 15-и дневным периодом.

In [11]:
# Calculate the stochastic oscillator
data["stochastic_k_5"], data["stochastic_d_5"] = talib.STOCH(data["high"], data["low"], data["close"], fastk_period=5, slowk_period=3, slowd_period=3)
data["stochastic_k_15"], data["stochastic_d_15"] = talib.STOCH(data["high"], data["low"], data["close"], fastk_period=15, slowk_period=3, slowd_period=3)
data['stochastic_k_ratio'] = data['stochastic_k_5'] / data['stochastic_k_15']
data['stochastic_d_ratio'] = data['stochastic_d_5'] / data['stochastic_d_15']

### Средний истинный диапазон (ATR)

Average True Range - распространенный технический индикатор, используемый для измерения волатильности на рынке, измеряется как скользящее среднее значение True Ranges. Более высокий ATR компании означает более высокую волатильность акций. Однако ATR в первую очередь используется для определения времени выхода или входа в сделку, а не направления торговли акциями.
Как определено выше, медленный ATR представляет собой 5-дневную скользящую среднюю, а быстрый ATR - 15-дневную скользящую среднюю.

In [12]:
data["ATR_5"] = talib.ATR(data["high"], data["low"], data["close"], timeperiod=5)
data["ATR_15"] = talib.ATR(data["high"], data["low"], data["close"], timeperiod=15)
data['ATR_Ratio'] = data['ATR_5'] / data['ATR_15']

### Индекс средней направленности (ADX)

Индекс средней направленности был разработан Уайлдером для оценки силы тренда в ценах на акции. 

Два его основных компонента, +DI и -DI, помогают определить направление тренда. Как правило, ADX 25 или выше указывает на сильный тренд, а ADX менее 20 - на слабый.

In [13]:
data['ADX_5'] = talib.ADX(data['high'], data['low'], data['close'], timeperiod=5)
data['ADX_15'] = talib.ADX(data['high'], data['low'], data['close'], timeperiod=15)

### Индекс относительной силы (RSI)

RSI - один из наиболее распространенных индикаторов импульса, призванный количественно оценить изменение цены и скорость этого изменения.

In [14]:
data['RSI_5'] = talib.RSI(data['close'], timeperiod=5)
data['RSI_15'] = talib.RSI(data['close'], timeperiod=15)
data['RSI_ratio'] = data['RSI_5']/data['RSI_15']

### Скорость изменения (Rate of Change)

Темп изменения - это индикатор импульса, который объясняет динамику цены по отношению к цене, зафиксированной периодом ранее.

In [15]:
data['ROC'] = talib.ROC(data['close'], timeperiod=15)

## Моделирование

In [61]:
data['target'] = np.where(data['close'].diff() > 1, True, False)
data['diff'] = data['close'].diff()
data.dropna(inplace=True)

In [64]:
data.head()

,date,open,high,low,close,volume,sma_5,sma_10,sma_15,sma_50,...,ATR_15,ATR_Ratio,ADX_5,ADX_15,RSI_5,RSI_15,RSI_ratio,ROC,target,diff
5800,2009-10-19,6.708929,6.785714,6.626786,5.732773,942230800,5.735672,5.738782,5.682852,5.345236,...,1.074176,1.010277,36.772550,25.937123,54.800309,62.738108,0.873477,1.993032,False,0.054652
5829,2009-10-20,7.164286,7.205357,7.066071,6.001507,1141039200,5.788452,5.765202,5.709785,5.365793,...,1.100737,1.056282,40.849290,26.987368,81.616693,72.965620,1.118564,7.217596,False,0.268734
5858,2009-10-21,7.125714,7.453929,7.115357,6.187505,1193726800,5.870763,5.809498,5.749179,5.391210,...,1.124182,1.085798,47.025504,28.711277,87.852087,77.537666,1.133025,10.558364,False,0.185998
5887,2009-10-22,7.310714,7.423214,7.232500,6.195959,791392000,5.959173,5.857598,5.798175,5.415300,...,1.131618,1.081328,51.966475,30.320259,88.081759,77.721149,1.133305,13.457879,False,0.008454
5916,2009-10-23,7.346429,7.350000,7.258214,6.157917,420786800,6.055132,5.898271,5.836502,5.436750,...,1.133112,1.067615,55.919252,31.821975,79.614845,74.776171,1.064709,10.297464,False,-0.038043


In [17]:
features = ['sma_5', 'sma_10', 'sma_15', 'sma_50', 'sma_200', 'sma_ratio_5_15', 'sma_ratio_10_50', 
            'sma_5_Volume', 'sma_10_Volume', 'sma_15_Volume', 'sma_50_Volume', 'sma_Volume_ratio_5_15', 'sma_Volume_ratio_10_50', 'upper_band',
            'middle_band', 'lower_band', 'RSI', 'macd', 'macd_signal', 'macd_hist','stochastic_k_5', 'stochastic_d_5', 'stochastic_k_15',
            'stochastic_d_15', 'stochastic_k_ratio', 'stochastic_d_ratio', 'ATR_5',
            'ATR_15', 'ATR_Ratio', 'ADX_5', 'ADX_15', 'RSI_5', 'RSI_15', 'RSI_ratio', 'ROC']

In [18]:
X = data[features]
y = data["target"]

In [44]:
n = 0.90
idx = int(X.shape[0]//(1/(1-n)))
train_index = X.shape[0] - idx
X_train, X_test = X[:train_index], X[train_index:]
y_train, y_test = y[:train_index], y[train_index:]

In [52]:
X_train.shape, X_test.shape

((2540, 35), (282, 35))

In [48]:
gbm = GradientBoostingClassifier(n_estimators=250, learning_rate=0.2, verbose=False).fit(X_train, y_train)
print(f'sklearn GB roc_auc_score: {roc_auc_score(y_test, gbm.predict_proba(X_test)[:, 1])}')

sklearn GB roc_auc_score: 0.6610343061955966


In [49]:
clf = LGBMClassifier(n_estimators=300, verbosity=-1)
clf.fit(X_train, y_train)

print(f'LightGBM roc_auc_score: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

LightGBM roc_auc_score: 0.6737782329180179


In [50]:
clf = CatBoostClassifier(iterations=300, logging_level='Silent')
clf.fit(X_train, y_train)

print(f'CatBoost roc_auc_score: {roc_auc_score(y_test, clf.predict_proba(X_test)[:, 1])}')

CatBoost roc_auc_score: 0.6862945895203959
